# San Francisco Housing Choice For Kiki

## Scenario: 
#### Kiki got a new job in  San Francisco City, so she will move to SF City. The public transportation in SF is great and Kiki has enough time on the commute. Thus she does not need to stay near the company. However, Kiki needs to take her dogs for a walk in the parks and she also likes to do jogging in the park. Therefore, she prefers a place with most parks nearby.

## Goal: 
#### Help Kiki find the place with most parks or gardens.

Library imported:

In [1]:
import pandas as pd
from urllib.request import urlopen
from six.moves import urllib
import json
from pandas.io.json import json_normalize

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

## 1.The Dataset

In [25]:
url = 'https://data.sfgov.org/api/views/a8z7-xscr/rows.json?accessType=DOWNLOAD'

In [26]:
json_obj = json.load(urllib.request.urlopen(url))

In [27]:
with open('SanFrancisco.Neighborhoods.json') as f:
      data = json.load(f)

In [28]:
# assign relevant part of JSON to venues
venues = data['features']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(5)

,id,type,geometry.geometries,geometry.type,properties.id,properties.neighborhood
0,94105,Feature,"[{'type': 'Polygon', 'coordinates': [[[-122.39...",GeometryCollection,94105,Rincon Hill
1,94107,Feature,"[{'type': 'Polygon', 'coordinates': [[[-122.38...",GeometryCollection,94107,South Beach
2,94108,Feature,"[{'type': 'Polygon', 'coordinates': [[[-122.40...",GeometryCollection,94108,Chinatown
3,94109,Feature,"[{'type': 'Polygon', 'coordinates': [[[-122.42...",GeometryCollection,94109,Nob Hill
4,94112,Feature,"[{'type': 'Polygon', 'coordinates': [[[-122.42...",GeometryCollection,94112,Ingleside


### Cleaning of the dataset:

In [29]:
def get_category_type(row):
    try:
        categories_list = row['geometry.geometries']
    except:
        categories_list = row['geometry.geometries']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['coordinates']

dataframe['coordinates'] = dataframe.apply(get_category_type, axis=1)

def get_category_type(row):
    try:
        categories_list = row['geometry.geometries']
    except:
        categories_list = row['geometry.geometries']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['type']

dataframe['coordinates_type'] = dataframe.apply(get_category_type, axis=1)

In [32]:
dataframe = dataframe.drop(columns=['geometry.geometries','id','type','geometry.type'])
dataframe = dataframe.rename(columns={'properties.id':'PostalCode','properties.neighborhood':'Neighborhood'})
dataframe = dataframe.sort_values(by='Neighborhood')

In [33]:
dataframe=dataframe[dataframe.coordinates_type!='MultiPolygon']

In [34]:
def avg(d):
    n=0
    x=0
    y=0
    for i in d:
        for j in i:
            x=j[0]+x
            y=j[1]+y
            n=n+1
    x=x/n
    y=y/n
    k=[x,y]
    return(k)

dataframe.coordinates=dataframe.coordinates.apply(avg)

In [35]:
dataframe['Latitude']= dataframe['coordinates'].apply(lambda x: x[1])
dataframe['Longitude']= dataframe['coordinates'].apply(lambda x: x[0])

In [36]:
SF_data = dataframe.drop(columns=['coordinates','coordinates_type'])

In [37]:
SF_data.head()

,PostalCode,Neighborhood,Latitude,Longitude
22,94134,Bayshore,37.718559,-122.412709
7,94124,Bayview,37.726869,-122.375455
5,94114,Castro,37.757239,-122.443479
2,94108,Chinatown,37.792016,-122.407486
15,94111,Financial District,37.801225,-122.399718
